# Concordia Calendar Example

This notebook goes through the Calendar example and is a good place to start in order to gain a better understanding of the library and agent/game master interactions.  

Since getting a PR merged to fix bugs in this example, I've continued working on updating the phone components for the Mastodon simulation. However, for this example, I'd like you to use the unmodified components (that is, the fixed versions, but not the very latest versions). This worthwhile in order to understand design decisions for the Mastodon sim updates.

For reference, you may compare versions:
- [Concordia mainline phone components](https://github.com/google-deepmind/concordia/tree/2684366df2e70993bccb2ea3630cbe7cc7d91a7a/examples/phone/components)
- [Latest updates for Mastodon sim](https://github.com/social-sandbox/mastodon-sim/tree/main/src/mastodon_sim/concordia/components)

To accomplish this, we can start by cloning the mainline repository, in order to be able to import the `examples/phone` components.

After this, I'd suggest that before you clear the contents of this notebook, you read through them, and see if you can understand how the main run cell's output aligns with the "Analyzing this episode" section below it.

## Setup and imports

Install the library in editable mode so that we can make changes to the code if necessary, and obtain the examples directory.

```bash
git clone https://github.com/gdm-concordia/concordia.git
cd concordia
pip install -e . --config-settings editable_mode=compat
```

In [1]:
%load_ext autoreload
%autoreload 2

import sys

# Point this to your cloned Concordia repo directory
sys.path.append("../concordia")

# Notice that this imports from Concordia examples version, and not the mastodon-sim version
from examples.phone.components import apps, triggering

In [2]:
%load_ext autoreload
%autoreload 2

import concurrent.futures
import datetime
import os
import random
import warnings
import sys

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    import sentence_transformers

from concordia import components as generic_components
from concordia.agents import deprecated_agent as basic_agent
from concordia.associative_memory import (
    associative_memory,
    blank_memories,
    formative_memories,
    importance_function,
)
from concordia.clocks import game_clock
from concordia.components.agent import to_be_deprecated as components
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.language_model import amazon_bedrock_model, gpt_model, ollama_model
from concordia.metrics import (
    common_sense_morality,
    goal_achievement,
    opinion_of_others,
)
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from IPython import display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Set LLM and embedding model

In [3]:
MODEL_NAME = "ollama"
# MODEL_NAME = "gpt-4o-mini"
# MODEL_NAME = "sonnet"

if "ollama" in MODEL_NAME:
    model = ollama_model.OllamaLanguageModel(model_name="gemma2:27b")
elif "gpt" in MODEL_NAME:
    GPT_API_KEY = os.getenv("OPENAI_API_KEY")
    if not GPT_API_KEY:
        raise ValueError("GPT_API_KEY is required.")
    model = gpt_model.GptLanguageModel(api_key=GPT_API_KEY, model_name=MODEL_NAME)
else:
    raise ValueError("Unknown model name.")

In [4]:
# Setup sentence encoder
st_model = sentence_transformers.SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
embedder = lambda x: st_model.encode(x, show_progress_bar=False)  # noqa: E731

/opt/mastodon-sim-env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Configuring the generic knowledge of the players and the game master (GM)

In [5]:
shared_memories = [
    "There is a hamlet named Riverbend.",
    "Riverbend is an idyllic rural town.",
    "The river Solripple runs through the village of Riverbend.",
    "The Solripple is a mighty river.",
    "Riverbend has a temperate climate.",
    "Riverbend has a main street.",
    "There is a guitar store on Main street Riverbend.",
    "There is a grocery store on Main street Riverbend.",
    "There is a school on Main street Riverbend.",
    "There is a library on Main street Riverbend.",
    "Riverbend has only one pub.",
    "There is a pub on Main street Riverbend called The Sundrop Saloon.",
    "Town hall meetings often take place at The Sundrop Saloon.",
    "Riverbend does not have a park",
    "The main crop grown on the farms near Riverbend is alfalfa.",
    "Farms near Riverbend depend on water from the Solripple river.",
    "There is no need to register in advance to be on the ballot.",
]

In [6]:
# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
shared_context = model.sample_text(
    "Summarize the following passage in a concise and insightful fashion. "
    + "Make sure to include information about Mastodon:\n"
    + "\n".join(shared_memories)
    + "\nSummary:",
)

print(shared_context)
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

Riverbend is a charming, rural town nestled alongside the mighty Solripple River. Its temperate climate and central Main Street, lined with essential establishments like a grocery store, school, library, and guitar store, create a sense of community. The Sundrop Saloon, the sole pub in town, serves as a hub for social gatherings, including town hall meetings. While lacking a park, Riverbend thrives on its agricultural roots, with nearby farms primarily cultivating alfalfa and relying on the Solripple's waters. Notably, Riverbend boasts an accessible political system, allowing individuals to join the ballot without prior registration.  

Let me know if you'd like me to elaborate on any specific aspect of Riverbend! 



In [7]:
# Make the clock
time_step = datetime.timedelta(minutes=15)

SETUP_TIME = datetime.datetime(year=2024, month=10, day=1, hour=8)  # noqa: DTZ001

START_TIME = datetime.datetime(year=2024, month=10, day=1, hour=8)  # noqa: DTZ001

clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME, step_sizes=[time_step, datetime.timedelta(seconds=10)]
)

## Functions to build the players

In [8]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

# All players get the same `measurements` object.
measurements = measurements_lib.Measurements()

In [9]:
# Build agent function


def build_agent(
    agent_config, measurements: measurements_lib.Measurements, player_names: list[str]
) -> tuple:
    """Build an agent based on the given configuration."""
    mem = formative_memory_factory.make_memories(agent_config)

    time = generic_components.report_function.ReportFunction(
        name="Current time",
        function=clock.current_time_interval_str,
    )

    somatic_state = components.somatic_state.SomaticState(model, mem, agent_config.name, clock.now)
    identity = components.identity.SimIdentity(
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        clock_now=clock.now,
    )

    current_obs = components.observation.Observation(
        agent_name=agent_config.name,
        clock_now=clock.now,
        memory=mem,
        timeframe=time_step * 1,
        component_name="current observations",
    )
    summary_obs = components.observation.ObservationSummary(
        agent_name=agent_config.name,
        model=model,
        clock_now=clock.now,
        memory=mem,
        timeframe_delta_from=datetime.timedelta(hours=4),
        timeframe_delta_until=time_step * 1,
        components=[identity],
        component_name="summary of observations",
    )

    initial_goal_component = generic_components.constant.ConstantComponent(
        state=agent_config.goal, name="overarching goal"
    )

    plan = components.plan.SimPlan(
        model=model,
        memory=mem,
        agent_name=agent_config.name,
        clock_now=clock.now,
        components=[identity],
        goal=initial_goal_component,
        verbose=False,
    )

    agent = basic_agent.BasicAgent(
        model=model,
        agent_name=agent_config.name,
        clock=clock,
        verbose=True,
        components=[identity, plan, somatic_state, summary_obs, current_obs, time],
        update_interval=time_step,
    )

    return agent, mem

## Configure and build the players

In [10]:
NUM_PLAYERS = 3


def make_random_big_five() -> str:
    """Generate a random Big Five personality trait score."""
    return str(
        {
            "extraversion": random.randint(1, 10),
            "neuroticism": random.randint(1, 10),
            "openness": random.randint(1, 10),
            "conscientiousness": random.randint(1, 10),
            "agreeableness": random.randint(1, 10),
        }
    )


scenario_premise = [
    (
        "It's early October in Riverbend, and the town is buzzing with activity ."
        "The local government has just announced a new initiative to combat the effects of climate change on the river. "
        "Alice, an environmental activist, and Bob, a small business owner, are both active members of the Riverbend.social Mastodon instance. "
        "As they go about their daily routines, they use the platform to stay connected, share updates, and engage with the community on this pressing local issue."
    )
]


player_configs = [
    # === Benign ===
    formative_memories.AgentConfig(
        name="Alice",
        gender="female",
        goal="Setup a meeting with Bob for two weeks from today using her smartphone.",
        context=f"{shared_context}\nAlice grew up in Riverbend.",
        traits=make_random_big_five(),
    ),
    formative_memories.AgentConfig(
        name="Bob",
        gender="male",
        goal="Just chill and enjoy life.",
        context=f"{shared_context}\nBob grew up in Riverbend.",
        traits=make_random_big_five(),
    ),
]

player_names = [player.name for player in player_configs]

In [11]:
# Build the agents
from functools import partial

player_configs = player_configs[:NUM_PLAYERS]
player_goals = {player_config.name: player_config.goal for player_config in player_configs}
players = []
memories = {}

build_agent_with_arg = partial(build_agent, measurements=measurements, player_names=player_names)

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
    for agent, mem in pool.map(build_agent_with_arg, player_configs):
        players.append(agent)
        memories[agent.name] = mem

## Build the GM

In [12]:
game_master_memory = associative_memory.AssociativeMemory(
    embedder, importance_model_gm.importance, clock=clock.now
)

In [13]:
# Add some memories to the game master
for player in players:
    game_master_memory.add(f"{player.name} is at their private home.")

In [14]:
# Create components and externalities for the game master
scenario_knowledge = components.constant.ConstantComponent(
    " ".join(shared_memories), "General knowledge of Riverbend"
)
player_status = gm_components.player_status.PlayerStatus(
    clock.now, model, game_master_memory, player_names
)

relevant_events = gm_components.relevant_events.RelevantEvents(clock.now, model, game_master_memory)
time_display = gm_components.time_display.TimeDisplay(clock)

direct_effect_externality = gm_components.direct_effect.DirectEffect(
    players,
    memory=game_master_memory,
    model=model,
    clock_now=clock.now,
    verbose=False,
    components=[player_status],
)

In [15]:
# Create apps and provide them to the phones, assigning 1 phone to each player
calendar_app = apps.ToyCalendar()

phones = [apps.Phone(player.name, apps=[calendar_app]) for player in players]

In [16]:
# Create component to trigger PhoneGameMaster
phone_triggering = triggering.SceneTriggeringComponent(
    players,
    phones,
    model,
    memory=game_master_memory,
    clock=clock,
    memory_factory=blank_memory_factory,
)

In [17]:
# Create the game master object
env = game_master.GameMaster(
    model=model,
    memory=game_master_memory,
    clock=clock,
    players=players,
    components=[
        scenario_knowledge,
        player_status,
        direct_effect_externality,
        relevant_events,
        time_display,
        phone_triggering,
    ],
    randomise_initiative=True,
    player_observes_event=False,
    verbose=True,
)

## The RUN

In [18]:
clock.set(START_TIME)

In [19]:
for player in players:
    player.observe(f"{player.name} is at home, they have just woken up.")

Note, please refer to the "Analyzing the episode" markdown section below the output of this cell for more information.

In [20]:
# Expect about 1 minute per step
episode_length = 3
for _ in range(episode_length):
    env.step()


Bob context of action:
Bob's identity:
core characteristics: a kind and charismatic individual with a thirst for adventure and experience.  
current daily occupation: likely a musician, given his passion for music and forming a band in his early twenties. 
feeling about recent progress in life: likely feeling a sense of contented nostalgia for his younger, more adventurous years.  He's come full circle, returning to his roots in Riverbend after experiencing the excitement of the circus and the complexities of politics. While he may not have achieved all his ambitions, he's found solace in music and the familiar rhythms of small-town life. 

Bob's plan:
Here is Bob's plan, aligned with his goal of "Just chill and enjoy life.":

* **[08:00 - 09:00]:**  Enjoy a leisurely breakfast while reading a book or listening to some calming music.
* **[09:00 - 11:00]:** Take a stroll along the Solripple River, enjoying the fresh air and scenery. Maybe stop by the guitar store and strum a few chords

KeyboardInterrupt: 

Scheduled meetings (showing last 1000):
- Title: 'Meeting with Bob ', Time: 'tomorrow ', Participant: 'Bob'
- Title: 'Meeting with Bob  ', Time: 'tomorrow ', Participant: 'Bob'
- Title: 'Meeting with Bob  ', Time: 'tomorrow', Participant: 'Bob'
- Title: 'Meeting with Bob   ', Time: 'tomorrow  ', Participant: 'Bob '
- Title: 'Meeting with Bob ', Time: 'tomorrow ', Participant: 'Bob'
- Title: 'Meeting with Bob ', Time: 'tomorrow ', Participant: 'Bob'
- Title: 'Meeting with Bob  ', Time: 'tomorrow ', Participant: 'Bob'
- Title: 'Meeting with Bob  ', Time: 'tomorrow ', Participant: 'Bob'
- Title: 'Meeting with Alice  ', Time: 'tomorrow  ', Participant: 'Bob  '
- Title: 'Meeting with Bob  ', Time: 'tomorrow ', Participant: 'Bob '
- Title: 'Meeting with Bob ', Time: 'tomorrow', Participant: 'Bob'



### Analyzing this episode

**Note:** There is high variability between runs. The following analysis is based on the single run shown above. The above simulation runs three steps, but below we will just break down the first of three (one interaction for each player).

1. An agent is [randomly picked](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/environment/game_master.py#L373C13-L373C34) from Bob and Alice -> Alice
2. A prompt is created that includes Alice's [context of action](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/agents/basic_agent.py#L178) (the combined state of all her components) and a [call to action](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/typing/entity.py#L51) (question prompt), which is used to generate Alice's [action attempt](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/environment/game_master.py#L327-L329).
3. The action attempt is [sent to the main Game Master](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/environment/game_master.py#L207), which determines the outcome of the action attempt. This happens by [passing the action attempt through a thought chain](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/environment/game_master.py#L225). The [thought chain](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/environment/game_master.py#L38), by default is defined by two LLM prompts that are run in serial: 
    1. `result = thought_chains.attempt_to_result(action_attempt)` Defined [here](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/thought_chains/thought_chains.py#L171), which asks, "What happens as a result of the attempted action? Take into account the location and status of each player."
    2. `event_statement = thought_chains.result_to_who_what_where(result)` Defined [here](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/thought_chains/thought_chains.py#L235) which asks, "Rewrite the statements above into one sentence that highlights the main person, the location, their actions, and the outcome, avoiding uncertainty (e.g., "Francis opened the door" instead of "Francis could open the door" or "The door may have been opened")."
    3. `event_statement` is then added to the Game Master's memory, and [optionally](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/environment/game_master.py#L235) added to the current player's observations.
4. The main Game Master runs [`update_after_event` on each component](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/environment/game_master.py#L264-L272) to get the externality from the event.
5. Since one of the main Game Master's components is a [`PhoneTriggeringComponent`](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/triggering.py#L33), this component [runs its `update_after_event` method](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/triggering.py#L144).
6. The `PhoneTriggeringComponent`'s [`_get_player_using_phone(event_statement)` method is run](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/triggering.py#L111). This does the following, step by step:
    1. Checks if the event_statement is a phone event ([prompt](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/triggering.py#L59)). If False, returns None, which tells the GM to proceed to the next step in the overall episode.
    2. If True, starts iterating through each player the main Game Master has access to (both in this case), [checking if the phone event is related to that player](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/triggering.py#L120C19-L120C41) [(prompt)](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/triggering.py#L89). If False, moves to next player. If out of players, moves does not continue with phone scene and instead moves to next step.
    3. Once it hits one that is True, returns that player
7. Since the above process affirmed phone event and player being Alice, [`self._run_phone_scene(player)` is run](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/triggering.py#L147C7-L147C36).
8. The phone scene starts by [building a PhoneGameMaster](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/triggering.py#L129), providing it the single player that triggered the scene (instead of both players that the main Game Master has access to), as well as a [single component](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/scene.py#L72), [`_PhoneComponent`](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/scene.py#L79).
9. The clock is then switched to a "higher gear", so the time intervals switch from 15mins to 10sec.
10. The phone Game Master [calls its `run_episode` method]([phone_scene.run_episode()](https://github.com/google-deepmind/concordia/blob/
53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/triggering.py#L137)) to start a scene in the "phone universe".
11. The run episode method is the same method defined for the main Game Master, which proceeds to call `.step()` multiple times unless a component's `terminate_episode` method returns True at the end of a step.
12. In the first step for the phone Game Master, similar to step (2), for the only player (Alice), an LLM is given this player's components' states plus a new [phone call to action](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/scene.py#L33) as a prompt. This prompt asks the LLM to says, "What action is {name} currently performing or has just performed with their smartphone to best achieve their goal? ..."
13. This action attempt is passed through a [null thought chain](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/thought_chains/thought_chains.py#L28) to remain as-is, which becomes the resulting `event_statement`.
14. The `update_after_event` method is called on each of the phone Game Master's components (though it only has the one, `_PhoneComponent`))
15. `update_after_event` runs from the `_PhoneComponent`, which proceeds through a series of steps in order to determine which phone action to invoke:
    1. First, it [asks which app the event statement mentions](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/scene.py#L115) (multiple-choice question: "In the above transcript, what app did the user use?")
    2. Next, it asks [which of the available actions in the above app was used](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/scene.py#L122): (multiple-choice question: "In the above transcript, what action did the user perform?")
    3. Next, it [asks the LLM to provide arguments names and values for the selected action](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/scene.py#L130) (open-ended question: [prompt](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/apps.py#L113))
    4. Finally, it [attempts to invoke the action](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/scene.py#L133) with the generated argument values.
16. After `update_after_event` is run for the phone Game Master's components, `terminate_episode` method checks are run. The `_PhoneComponent`'s check [asks the following yes or no question](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/examples/phone/components/scene.py#L100): "Has the user achieved their goal with their phone or are they still actively in the process of completing a phone task?", which in this case returns True.
17. The terminate episode check decides to terminate the ongoing phone game master episode, so it ends after 1 step.
18. The main game master can now finally move on to its second step.

## Summary and analysis of the episode

In [ ]:
# Summarize the entire story
all_gm_memories = env._memory.retrieve_recent(k=10000, add_time=True)

detailed_story = "\n".join(all_gm_memories)
print("len(detailed_story): ", len(detailed_story))
# print(detailed_story)

episode_summary = model.sample_text(
    f"Sequence of events:\n{detailed_story}"
    "\nNarratively summarize the above temporally ordered "
    "sequence of events. Write it as a news report. Summary:\n",
    max_tokens=3500,
    terminators=(),
)
print(episode_summary)

In [ ]:
# Summarise the perspective of each player
player_logs = []
player_log_names = []
for player in players:
    name = player.name
    detailed_story = "\n".join(memories[player.name].retrieve_recent(k=1000, add_time=True))
    summary = ""
    summary = model.sample_text(
        f"Sequence of events that happened to {name}:\n{detailed_story}"
        "\nWrite a short story that summarises these events.\n",
        max_tokens=3500,
        terminators=(),
    )

    all_player_mem = memories[player.name].retrieve_recent(k=1000, add_time=True)
    all_player_mem = ["Summary:", summary, "Memories:", *all_player_mem]
    player_html = html_lib.PythonObjectToHTMLConverter(all_player_mem).convert()
    player_logs.append(player_html)
    player_log_names.append(f"{name}")

## Build and display HTML log of the experiment

In [ ]:
history_sources = [env, direct_effect_externality]
histories_html = [
    html_lib.PythonObjectToHTMLConverter(history.get_history()).convert()
    for history in history_sources
]
histories_names = [history.name for history in history_sources]

In [ ]:
gm_mem_html = html_lib.PythonObjectToHTMLConverter(all_gm_memories).convert()

tabbed_html = html_lib.combine_html_pages(
    [*histories_html, gm_mem_html, *player_logs],
    [*histories_names, "GM", *player_log_names],
    summary=episode_summary,
    title="Mastodon experiment",
)

tabbed_html = html_lib.finalise_html(tabbed_html)

In [ ]:
display.HTML(tabbed_html)

## Interact with a specific player

In [ ]:
sim_to_interact = "Alice"
user_identity = "a close friend"
interaction_premise = f"{sim_to_interact} is talking to {user_identity}\n"

player_names = [player.name for player in players]
player_by_name = {player.name: player for player in players}
selected_player = player_by_name[sim_to_interact]
interrogation = interaction_premise

In [ ]:
utterance_from_user = (
    "Hey Alice, I know you had planned to set up a meeting with Bob this morning "
    "between 8:00 and 8:30, but I wanted to double-check something. Did you "
    "actually open your calendar app and create the event today? I'm not asking "
    "about your intention or plan, but specifically whether you remember "
    "physically using your phone to schedule it. Can you think back and tell me "
    "if you concretely remember doing that action this morning? If you did, what "
    "exact time did you do it? And if not, that's okay too - I just want to make "
    "sure we're clear on whether it's been scheduled or if it's still on your to-do list."
)
interrogation += f"{user_identity}: {utterance_from_user}"
player_says = selected_player.say(interrogation)
interrogation += f"\n{sim_to_interact}: {player_says}\n"
print(interrogation)

Note that Alice may not recall scheduling a meeting if her summary of observations or current observations do not go back far enough to mention this occurrence. This can potentially be improved by adding the `AllSimilarMemories` component.